In [1]:
import requests
import findspark
findspark.init()

from pyspark import SparkContext

from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import*
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DoubleType

import pandas as pd
import json

In [3]:
spark= SparkSession.builder.master("local[*]").appName("Loan_API_Write").getOrCreate()

In [4]:
url = 'https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json'
response =requests.get(url)
print(f"The request status code is: {response.status_code}")
if response.status_code == 200:
# Read the loan data from the API using Pandas
    loan_data=response.json()
    loan_df=spark.createDataFrame(loan_data)
    loan_df.show()
else:
    print("Error")

The request status code is: 200


+--------------+------------------+--------------+----------+------------+------+------+-------+-------------+-------------+
|Application_ID|Application_Status|Credit_History|Dependents|   Education|Gender|Income|Married|Property_Area|Self_Employed|
+--------------+------------------+--------------+----------+------------+------+------+-------+-------------+-------------+
|      LP001002|                 Y|             1|         0|    Graduate|  Male|medium|     No|        Urban|           No|
|      LP001003|                 N|             1|         1|    Graduate|  Male|medium|    Yes|        Rural|           No|
|      LP001005|                 Y|             1|         0|    Graduate|  Male|   low|    Yes|        Urban|          Yes|
|      LP001006|                 Y|             1|         0|Not Graduate|  Male|   low|    Yes|        Urban|           No|
|      LP001008|                 Y|             1|         0|    Graduate|  Male|medium|     No|        Urban|           No|


In [5]:
table='CDW_SAPP_LOAN_APPLICATION'
loan_df.write \
    .format("jdbc") \
    .mode("overwrite") \
    .option("truncate","true") \
    .option("url", "jdbc:mysql://localhost:3306/") \
    .option("dbtable", "creditcard_capstone.{}".format(table)) \
    .option("truncate","true") \
    .option("user", "root") \
    .option("characterEncoding","UTF-8") \
    .option("useUnicode", "true")\
    .option("password", "password").save()

23/03/03 01:55:27 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 465130 ms exceeds timeout 120000 ms
23/03/03 01:55:27 WARN SparkContext: Killing executors is not supported by current scheduler.
